<a href="https://colab.research.google.com/github/KnowledgeLab/Thinking-With-Deep-Learning-2026/blob/main/Tutorials-Homework_Notebooks/Week%205/Week_5_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Thinking with Deep Learning: Week 5 Homework Modules**

## Sampling, Fine Tuning, Benchmarking, and Tools

- __Instructor:__ James Evans

- __Notebook Author & TAs:__ Avi, Gio, Jesse, Shiyang 

This week we explore the potential for data-driven bias in social scientific inference with AI agents, and how to fine-tune large transformer-based agents to limit bias or adopt specific perspectives.

__Perform 3 out of this week's following 4 modules:__

##1. [**Sampling**](#module1)
### **Summary:**
Sampling is a critical method for data science and AI. In this section we review probabilistic and non-probabilistic sampling techniques, handling imbalanced datasets, and bootstrap sampling for testing embedding stability.

### **Tasks/Questions:**
**1)** Run 3 probabilistic sampling methods and 2 non-probabilistic methods and explore the samples returned.

**2)** Find an imbalanced dataset and build a classifier to predict the label causing the imbalance. Explore undersampling and oversampling solutions.

**3)** Use bootstrap sampling to test the stability of a word, sentence, or graph embedding.

##2. [**Fine-Tuning with LoRA and QLoRA**](#module2)
### **Summary:**
Fine-tuning neural networks involves taking a pre-trained model and adjusting its parameters for a specific task. We focus on Low-Rank Adaptation (LoRA) and Quantized LoRA (QLoRA) for efficient fine-tuning of large language models.

### **Tasks/Questions:**
**1)** Fine-tune a small LLM (e.g., GPT-2) using LoRA on a text dataset of your choice.

**2)** Compare the number of trainable parameters between full fine-tuning and LoRA.

**3)** Experiment with different LoRA rank values (r=4, 8, 16) and observe the impact on performance.

##3. [**Benchmarking LLM Agents**](#module3)
### **Summary:**
Evaluating LLM-based agents requires systematic benchmarking approaches. We explore Centered Kernel Alignment (CKA) for comparing neural representations and discuss frameworks for evaluating agent capabilities, safety, and performance.

### **Tasks/Questions:**
**1)** Implement CKA to compare representations between two different models or layers.

**2)** Design a simple benchmark task for an LLM agent and evaluate its performance.

**3)** Discuss potential biases in benchmark design and how they might affect evaluation results.

##4. [**Tools and the Model Context Protocol (MCP)**](#module4)
### **Summary:**
Modern LLM agents interact with external tools and data sources. The Model Context Protocol (MCP) provides a standardized way for AI models to access context from various sources. We explore tool use patterns and MCP implementation.

### **Tasks/Questions:**
**1)** Implement a simple tool-using agent that can call external functions.

**2)** Create an MCP server that exposes a data source to an LLM.

**3)** Discuss the implications of tool use for AI agent safety and capability.

In [ ]:
# @markdown Mark the Modules you completed
Sampling = False  # @param {type:"boolean"}
FineTuning_LoRA = False  # @param {type:"boolean"}
Benchmarking_LLM_Agents = False  # @param {type:"boolean"}
Tools_and_MCP = False  # @param {type:"boolean"}

# Module 1: Sampling


[Sampling](https://en.wikipedia.org/wiki/Sampling_(statistics)) is a critical method which you've likely come across in previous data or social science explorations. In this section we will review several of the more popular techniques used in research and industry. There is no one standard package for sampling with python, and we will be using the rich PyData ecosystem in different ways to achieve our aims.

Here are some links you may wish to explore:

- Krippendorff, Klaus. 2004. Content Analysis: An Introduction to its Methodology. Thousand Oaks, CA: Sage: [“Sampling”](https://canvas.uchicago.edu/courses/33672/files/4767016/download?wrap=1)(for sampling content).
- [Data Scientist's guide to 8 sampling techniques](https://www.analyticsvidhya.com/blog/2019/09/data-scientists-guide-8-types-of-sampling-techniques/
)
- [KDnuggets - 5 sampling algorithms](https://www.kdnuggets.com/2019/09/5-sampling-algorithms.html)

Sampling procedures are often divided into probabilistic and non-probabilistic methods, and we begin with the same division before jumping into methods tuned for maximizing machine and deep learning model generalizability.




In [ ]:
# Module 1 Setup: Install required packages and download data
# Note: We install packages carefully to avoid pandas version conflicts in Colab

!pip install scikit-learn imbalanced-learn gensim yellowbrick gdown -q
!pip install littleballoffur --no-deps -q  # Install without dependencies to avoid pandas downgrade
!pip install networkx scipy decorator -q   # Install littleballoffur's actual dependencies

# Download the mental health dataset from Google Drive
import gdown
gdown.download('https://drive.google.com/uc?id=1-0r2C4z9-vAedJ4uum-TfI4Zy4gKDGaQ', '/content/mental health.csv', quiet=False)

print('Module 1 setup complete!')

## Probabilistic Sampling

Often we hear about sampling from a probability distribution, but such methods typically extend to sampling from real world data. For the examples in this section, we use both a real dataset and randomly constructed distributions, both of which are useful tools. The power of these methods in the context of machine and deep learning (and its human and social uses!) will become clear.  

### Dataset

One of the datasets we will be using is the "Mental Health in Tech Survey" data, an open source survey data about mental health conditions of workers in tech industries. We also worked with this data for the week 5 hint. You can find the data at Kaggle:

https://www.kaggle.com/osmi/mental-health-in-tech-survey

[google drive link of cleaned data](https://drive.google.com/file/d/1-0r2C4z9-vAedJ4uum-TfI4Zy4gKDGaQ/view?usp=sharing)

We provided a cleaned version. The predictors contain 1 continuous variable (age), 3 dummies (Do you work remotely? Is your employer primarily a tech company? Does your employer provide any mental health benefits?) and 2 categorical variables (gender-male/female/other; can you discuss your mental health issue with supervisors-yes/sometimes/no).

The outcome is an answer to the question: If you have a mental health condition, do you feel it interferes with your work? The DV is measured with a 5-categorical variable: NA (no mental health condition), never, rarely, sometimes, often.

**An important note**: very often we are sampling from a very large dataset or population - in this case, our dataset is small enough that we can analyze the whole dataset (which itself represents a sample of the tech population), and the sampling is purely illustratory.



In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
df = pd.read_csv('/content/mental health.csv')
df.head()

In [ ]:
df

,age,remote,benefits,tech,gender,supervisor,interfere
0,8,1,1,1,other,yes,4
1,21,0,0,1,other,some of them,3
2,32,0,0,1,other,no,4
3,28,0,0,1,other,some of them,2
4,27,1,1,1,other,yes,4
...,...,...,...,...,...,...,...
1254,32,1,1,1,other,yes,4
1255,26,1,0,1,other,some of them,3
1256,30,0,1,1,other,yes,2
1257,18,1,1,1,other,yes,0


### Random



In [ ]:
sample_df = df.sample(100)

In [ ]:
sample_df

,age,remote,benefits,tech,gender,supervisor,interfere
1237,37,0,1,1,male,yes,3
1010,44,1,0,0,male,no,0
676,27,0,0,1,male,no,0
1202,35,1,0,0,male,no,3
1234,40,0,0,1,male,yes,3
...,...,...,...,...,...,...,...
1209,34,1,0,1,male,yes,0
492,50,1,0,1,male,yes,1
853,25,0,0,1,male,yes,0
190,33,1,0,1,male,yes,3


[SciPy](https://docs.scipy.org/doc/numpy-1.10.1/reference/routines.random.html) and [numpy](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) also offer many popular sampling techniques, usually to be applied on a list or array, or sampled from a distribution.

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
random_vals = np.random.rand(500)

In [ ]:
random_vals

array([0.87756341, 0.08953556, 0.22070822, 0.13464803, 0.77722257,
       0.28488422, 0.56600295, 0.22611083, 0.48242142, 0.4443197 ,
       0.98419314, 0.96476736, 0.79155608, 0.37454349, 0.18619156,
       0.67029591, 0.40881112, 0.03007655, 0.8814407 , 0.61269684,
       0.14286068, 0.35440079, 0.58676128, 0.99899733, 0.96259004,
       0.0755457 , 0.36263351, 0.11824383, 0.9028245 , 0.18863398,
       0.34411585, 0.83408633, 0.42978301, 0.67239463, 0.4870574 ,
       0.87684776, 0.3114396 , 0.96132891, 0.41435173, 0.63550723,
       0.60379586, 0.97713603, 0.07941813, 0.12634668, 0.03684195,
       0.51592944, 0.04105821, 0.72068143, 0.49918895, 0.67089092,
       0.04196314, 0.05935925, 0.39744373, 0.20719472, 0.27868948,
       0.88444396, 0.52586205, 0.02107223, 0.33888305, 0.01135199,
       0.1468008 , 0.68537626, 0.30995161, 0.26176913, 0.85267301,
       0.92495817, 0.25419009, 0.60623648, 0.39284497, 0.88388632,
       0.44423055, 0.17403786, 0.46351792, 0.127054  , 0.52500

In [ ]:
np.random.choice(random_vals, 10)

array([0.37454349, 0.33888305, 0.97607187, 0.86276743, 0.21395688,
       0.61568858, 0.28488422, 0.86564232, 0.18907974, 0.10045201])

In this case, we first generated an array of size 500 by randomly sampling between 0 and 1, and then sampled 10 values from this list. We can similarly sample values from any list by using the scipy and numpy random module.

### Stratified

Stratified random sampling is a method for sampling that involves the division of a population into smaller sub-groups known as strata and then sampling elements equally across those groups. In stratified random sampling, or stratification, the strata are formed based on members' shared attributes or characteristics such as age, income level, or onsite vs. remote work status. [Here](https://www.investopedia.com/terms/stratified_random_sampling.asp) is a source for reading more about it. We use stratified sampling in order to balance our dataset according to attributes of interest. Deep learning models trained to predict very rare classes, for example, may intelligently predict their absence and achieve high performance scores if we do not balance the data with respect to classes of interest.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
stratified_sample, _ = train_test_split(df, train_size=0.10, stratify=df[['remote']])

In [ ]:
stratified_sample

,age,remote,benefits,tech,gender,supervisor,interfere
159,26,1,1,1,female,no,2
1069,34,0,1,1,male,yes,4
540,22,0,1,1,male,yes,0
133,32,0,1,1,female,some of them,1
432,25,0,0,1,male,some of them,3
...,...,...,...,...,...,...,...
1056,34,0,1,1,male,no,3
991,38,0,1,1,female,no,0
720,28,0,1,1,male,no,2
333,25,1,0,1,male,yes,0


In this case we get stratified results for remote work, and our remote work attribute is represented as per the original ratio.

### Varying probability sampling (weighted by variables of interest)

We can also sample by weighting certain attributes so that our sample represents them proportional to those weights. With Pandas we can do this using a DataFrame column as weights. Rows with larger value in the column are more likely to be sampled.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html


In [ ]:
weight_sample = df.sample(n=125, weights='age')

In [ ]:
weight_sample

,age,remote,benefits,tech,gender,supervisor,interfere
1096,32,0,0,1,male,some of them,1
411,24,0,1,1,male,yes,2
425,43,1,1,1,male,some of them,3
1066,34,0,0,1,male,yes,4
872,38,1,1,1,male,some of them,3
...,...,...,...,...,...,...,...
389,38,0,0,1,male,no,3
798,39,0,1,1,male,some of them,4
1126,46,0,1,1,male,no,3
1059,24,0,0,1,male,yes,1


In [ ]:
weight_sample['age'].mean()

32.744

In [ ]:
df['age'].mean()

32.01906274821287

Because we weighted by age, older ages are prioritised, which leads to that small bump in the mean age of the weighted sample.

### Cluster Sampling (lists of large groups of units)

In cluster sampling, the sampling unit is the whole cluster; Instead of sampling individuals from within each group, a researcher will study whole clusters.

The difference between cluster and stratified sampling is that with cluster sampling, you have natural groups separating your population. For example, you might be able to divide your data into natural groupings like city blocks, voting districts or school districts.

In short, the population is divided into subsets or subgroups that are considered as clusters, and from the numbers of clusters, we select the individual cluster for the next step to be performed.

You can read more about cluster sampling [here](https://www.geeksforgeeks.org/cluster-sampling-in-pandas/).

There are a couple of ways to do cluster sampling - one way is to meaningfully partition or cluster the dataset, and then choose that whole cluster as your sample. In this case, we will cluster based on age and then choose samples from one of these clusters. For some purposes (e.g., musical tastes), age would be poor variable on which to sample, naturally creating clustered results.


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
age_cluster = KMeans(n_clusters=12)

In [ ]:
age_cluster.fit(np.array(df['age']).reshape(-1, 1))

KMeans(n_clusters=12)

In [ ]:
df['cluster'] = age_cluster.labels_

In [ ]:
df

,age,remote,benefits,tech,gender,supervisor,interfere,cluster
0,8,1,1,1,other,yes,4,11
1,21,0,0,1,other,some of them,3,7
2,32,0,0,1,other,no,4,9
3,28,0,0,1,other,some of them,2,5
4,27,1,1,1,other,yes,4,5
...,...,...,...,...,...,...,...,...
1254,32,1,1,1,other,yes,4,9
1255,26,1,0,1,other,some of them,3,5
1256,30,0,1,1,other,yes,2,0
1257,18,1,1,1,other,yes,0,7


In [ ]:
df[df['cluster'] == 3]

,age,remote,benefits,tech,gender,supervisor,interfere,cluster
15,47,1,0,1,female,no,0,3
171,48,0,0,1,other,some of them,4,3
208,50,0,1,1,male,yes,0,3
224,47,0,1,1,male,yes,3,3
229,48,0,0,1,male,no,1,3
244,48,0,0,1,male,no,0,3
263,51,1,0,1,male,no,1,3
435,51,1,0,1,male,no,3,3
476,51,1,1,1,male,yes,1,3
492,50,1,0,1,male,yes,1,3


In [ ]:
df[df['cluster'] == 3].sample(10)

,age,remote,benefits,tech,gender,supervisor,interfere,cluster
171,48,0,0,1,other,some of them,4,3
557,49,0,1,1,male,yes,1,3
15,47,1,0,1,female,no,0,3
208,50,0,1,1,male,yes,0,3
229,48,0,0,1,male,no,1,3
1073,49,1,0,1,male,no,3,3
224,47,0,1,1,male,yes,3,3
1020,51,0,0,1,male,some of them,1,3
1121,50,1,0,1,male,yes,3,3
435,51,1,0,1,male,no,3,3


### Systematic

Systematic Sampling is defined as a type of Probability Sampling where a researcher can select targeted data from large set of data. Targeted data is chosen by selecting random starting points and from those adding others following a certain interval (e.g., randomly selecting Thursday papers from the *New York Times*). In this way a small subset (sample) is extracted from large data.

You can read more about systematic sampling [here](https://www.geeksforgeeks.org/systematic-sampling-in-pandas/).

For an example using our current dataset, suppose we sample from only those who work remotely.

In [ ]:
remote_sample = df[df['remote'] == 1].sample(10)

In [ ]:
remote_sample

,age,remote,benefits,tech,gender,supervisor,interfere,cluster
313,27,1,0,1,male,some of them,3,5
806,38,1,0,1,male,yes,0,1
4,27,1,1,1,other,yes,4,5
401,30,1,0,1,male,some of them,1,0
813,33,1,0,1,male,yes,4,9
1161,18,1,0,0,male,no,0,7
533,35,1,0,1,male,no,0,4
1057,33,1,0,1,male,no,4,9
259,40,1,1,1,male,no,3,1
816,33,1,0,1,male,no,2,9


### Graph Sampling

Sometimes we come across social data that must be sampled as a graph to retain its integrity. In this section we will explore some ways of sampling from graphs to minimize poor prediction (and biased inferences). To illustrate these examples, we will sample from a graph-based dataset. It is possible to extend these methods to your data that might not be (at first) represented in a graph-like way by restructuring your data in a way that supports these methods (e.g., recall that structured data can be interpreted as an adjacency matrix).

We will use a python package we briefly touched upon earlier, [Little Ball of Fur](https://arxiv.org/pdf/2006.04311.pdf).

#### Data

We will use a dataset based on GitHub data. In this graph nodes represent GitHub developers and edges between them are mutual follower relationships. For details about the dataset see this [paper](https://arxiv.org/abs/1909.13021Z).

In [ ]:
!pip install littleballoffur

     |████████████████████████████████| 3.1 MB 34.1 MB/s 
  Created wheel for littleballoffur: filename=littleballoffur-2.1.12-py3-none-any.whl size=40409 sha256=6d1ed8a787d016a21166a60a9749189886b0af069650e8162c12180e3e3d70a3
  Stored in directory: /root/.cache/pip/wheels/44/ba/f8/f66537badaf1e475ec56de5af58ce294bbb0fbe570ca888f8e
  Created wheel for networkit: filename=networkit-7.1-cp37-cp37m-linux_x86_64.whl size=8049002 sha256=abfced58b80726af2f5b00ccd261ca0550b63849d446b777f73c2363f1b18a83
  Stored in directory: /root/.cache/pip/wheels/a4/8f/06/512044bbf7240e78fc054b506e075f0871c7642bec400a2647
Successfully built littleballoffur networkit


In [ ]:
from littleballoffur import GraphReader

reader = GraphReader("github")

graph = reader.get_graph()

#### Node Sampling

We will first look at some popular node sampling algorithms. Let’s use the PageRank Proportional Node Sampling method from [Sampling From Large Graphs](https://cs.stanford.edu/people/jure/pubs/sampling-kdd06.pdf). We will sample approximately 50% of the original nodes from the network.

In [ ]:
from littleballoffur import PageRankBasedSampler, RandomNodeSampler


In [ ]:
number_of_nodes = int(0.5*graph.number_of_nodes())

In [ ]:
pagerank_sampler = PageRankBasedSampler(number_of_nodes = number_of_nodes)


In [ ]:
randomnode_sampler = RandomNodeSampler(number_of_nodes = number_of_nodes)

In [ ]:
randomnodes_graph = pagerank_sampler.sample(graph)

In [ ]:
pagerank_graph = pagerank_sampler.sample(graph)

#### Sub-graph Sampling

We also look at a series of sampling algorithms that sample sub-graphs from larger graphs.

First is an implementation of node sampling by random walks--a simple random walker that creates an induced subgraph by wandering around.

In [ ]:
from littleballoffur import RandomWalkSampler

In [ ]:
randomwalk_sampler = RandomWalkSampler(number_of_nodes=number_of_nodes)

In [ ]:
randomwalk_graph = randomnode_sampler.sample(graph)

Now let’s use the Metropolis-Hastings Random Walk Sampler method from [Metropolis Algorithms for Representative Subgraph Sampling](https://ieeexplore.ieee.org/document/4781123).  The random walker has a probabilistic acceptance condition for adding new nodes to the sampled node set. This constraint can be parametrized by the rejection constraint exponent.

In [ ]:
from littleballoffur import MetropolisHastingsRandomWalkSampler

In [ ]:
mhrw_sampler = MetropolisHastingsRandomWalkSampler(number_of_nodes = number_of_nodes)


In [ ]:
mhrw_graph = mhrw_sampler.sample(graph)

You will find many other sub-graph sampling algorithms, and many variations of the Random Walk algorithm (RandomWalkWithJumpSampler, RandomNodeNeighborSampler, RandomWalkWithRestartSampler) in the [Exploration Sampling](https://little-ball-of-fur.readthedocs.io/en/latest/modules/exploration_sampling.html).


## Non-probabilistic Sampling and Extensions

A lot of the sampling we will see in this section involves sampling over a network or graph. For example, in a survey, we may ask one respondent to choose the next, and in that way create a sub-graph within the full social graph from which we seek to sample.


### Snowball Sampling

Snowball sampling is where research participants recruit others for a test or study. It is used where potential participants are hard to find. It’s called snowball sampling because (in theory) once you have the ball rolling, it picks up more “snow” along the way and becomes larger and larger. Snowball sampling is a non-probability sampling method. ([link to explanation](https://www.statisticshowto.com/probability-and-statistics/statistics-definitions/snowball-sampling/))

If we manually inspect the sampling, we can add our own criteria for stopping; in the package implementation, it stops when we have enough samples.


In [ ]:
from littleballoffur import SnowBallSampler

In [ ]:
snowball_sampler = SnowBallSampler(number_of_nodes = number_of_nodes)


In [ ]:
snowball_graph = snowball_sampler.sample(graph)

littleballoffur also offers stochastic extensions of the Snowball Sampler. The Forest Fire Sampler is a stochastic snowball sampling method where the expansion is proportional to the burning probability, described [here](https://cs.stanford.edu/people/jure/pubs/sampling-kdd06.pdf).

In [ ]:
from littleballoffur import ForestFireSampler, SpikyBallSampler

In [ ]:
forestfire_sampler = ForestFireSampler(number_of_nodes=number_of_nodes)

In [ ]:
forestfire_graph = forestfire_sampler.sample(graph)

Below we use spiky ball sampling. The procedure is a filtered breadth-first search sampling method where the expansion is performed over a random subset of neighbors. Originally described [here](https://www.mdpi.com/1999-4893/13/11/275).

In [ ]:
spikyball_sampler = SpikyBallSampler(number_of_nodes=number_of_nodes)

In [ ]:
spikyball_graph = spikyball_sampler.sample(graph)

We note that [Respondent-driven sampling](http://www.respondentdrivensampling.org/) and the [Network Scale-up method](https://journals.sagepub.com/doi/full/10.1177/0081175016665425) extend the snowball to improve inference.

That's the last of our graph based sampling. We will now discuss some other methods used. Note that the following sections are theoretical with no code, but the previous code can help you implement these methods.

### Convenience

This simply refers to using a sample that is at-hand and easy to analyse. For example, you may choose a free sub-graph of the full Facebook friend graph to run experiments because you do not have access to more data. You can refine your methods on this data before moving on to a larger dataset.

You can read more about it [here](https://methods.sagepub.com/reference/encyclopedia-of-survey-research-methods/n105.xml).

In [ ]:
# empty cell

### Quota Sampling

Very similar to stratified sampling, but we may not randonly sample from the strata and choose the whole sample according to these desired traits or qualities. Here are some links to read more:

- [QuestionPro: Quota Sampling definition](https://www.questionpro.com/blog/quota-sampling/#:~:text=Quota%20sampling%20is%20defined%20as,to%20specific%20traits%20or%20qualities.)
- [Statistics How To: Quota Sampling](https://www.statisticshowto.com/quota-sampling/)
- [humansofdata: Quota Sampling](https://humansofdata.atlan.com/2016/04/quota-sampling-when-to-use-how-to-do-correctly/)


In [ ]:
# empty cell

### Purposive / Relevance / Judgement sampling

[A useful blog post](https://www.alchemer.com/resources/blog/purposive-sampling-101/)

[Paper: Purposeful sampling for qualitative data collection and analysis in mixed method implementation research](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4012002/)

Purposive sampling, also known as judgmental, selective, or subjective sampling, is a form of non-probability sampling in which researchers rely on their own judgment when choosing members of the population to participate in their surveys.

This survey sampling method requires researchers to have prior knowledge about the purpose of their studies so that they can properly choose and approach eligible participants for surveys conducted.

Researchers use purposive sampling when they want to access a particular subset of people, as all participants of a survey are selected because they fit a particular profile.

In [ ]:
# empty cell

## Sampling Imbalanced Classes

A common problem we deal with in real world datasets is imbalance, when we have one (or a few) class, category or sections that have higher or lower representation than the other classes. This situation is often referred to as having imbalanced classes, and the python package [imbalanced-learn](https://github.com/scikit-learn-contrib/imbalanced-learn) is specifically built to tackle this, although one can frame this as a stratified sampling problem, as described above.

In this section we will be drawing on a tutorial by [Investigate.ai](https://investigate.ai/), a school for teaching data science to journalists. The tutorial we will follow is in this [repository](https://github.com/littlecolumns/ds4j-notebooks/blob/master/classification/notebooks/Correcting%20for%20imbalanced%20datasets.ipynb).



### Classification problems with imbalanced inputs

Oftentimes when we're doing real-world classification, we have the problem of **"imbalanced classes"**.

Let's say we're analyzing a document dump, and trying to find documents interesting to us. Maybe we're only interested in 10% of them! The fact that there's such a bias - 90% are uninteresting - **can damage the accuracy of our classifier.** Let's take a look at [imbalanced-learn](https://imbalanced-learn.readthedocs.io/en/stable/) library to help address this challenge!

### Prep work: Downloading necessary files
First, we need to download the following data.
* **recipes-indian.csv:** Indian classification recipes - a selection of recipe ingredient lists, half of them being labeled as Indian cuisine.
* **recipes.csv:** recipes - a selection of recipe ingredient lists, each labeled with the cuisine from which it hearkens.


In [ ]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/classification/data/recipes-indian.csv -P data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/classification/data/recipes.csv -P data

--2022-04-17 04:34:12--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/classification/data/recipes-indian.csv
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1033379 (1009K) [text/csv]
Saving to: ‘data/recipes-indian.csv’

recipes-indian.csv  100%[===================>]   1009K  --.-KB/s    in 0.04s   

2022-04-17 04:34:12 (23.7 MB/s) - ‘data/recipes-indian.csv’ saved [1033379/1033379]

--2022-04-17 04:34:12--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/classification/data/recipes.csv
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6483086 (6.2M) [text/csv]
Saving to: ‘data/recip

You should be familiar with vectorizing, classification, and confusion matrices going in.

In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### Our datasets

We're going to be looking at two datasets: They're both **recipes and ingredient lists**, and with both we're predicting whether we can **accurately determine which recipes are Indian**.

Let's read both in.

In [ ]:
df_balanced = pd.read_csv("data/recipes-indian.csv")
df_balanced['is_indian'] = (df_balanced.cuisine == "indian").astype(int)

df_balanced.head()

,cuisine,id,ingredient_list,is_indian
0,indian,23348,"minced ginger, garlic, oil, coriander powder, ...",1
1,indian,18869,"chicken, chicken breasts",1
2,indian,36405,"flour, rose essence, frying oil, powdered milk...",1
3,indian,11494,"soda, ghee, sugar, khoa, maida flour, milk, oil",1
4,indian,32675,"tumeric, garam masala, salt, chicken, curry le...",1


In [ ]:
df_unbalanced = pd.read_csv("data/recipes.csv")
df_unbalanced['is_indian'] = (df_unbalanced.cuisine == "indian").astype(int)

df_unbalanced.head()

,cuisine,id,ingredient_list,is_indian
0,greek,10259,"romaine lettuce, black olives, grape tomatoes,...",0
1,southern_us,25693,"plain flour, ground pepper, salt, tomatoes, gr...",0
2,filipino,20130,"eggs, pepper, salt, mayonaise, cooking oil, gr...",0
3,indian,22213,"water, vegetable oil, wheat, salt",1
4,indian,13162,"black pepper, shallots, cornflour, cayenne pep...",1


They both look similar enough, right? A list of ingredients and an `is_indian` target column we'll use as our label.

### Finding the imbalance

The real difference is how many recipes are Indian in each dataset. Let's take a look:

In [ ]:
df_balanced.is_indian.value_counts()

1    3000
0    3000
Name: is_indian, dtype: int64

In [ ]:
df_unbalanced.is_indian.value_counts()

0    36771
1     3003
Name: is_indian, dtype: int64

Ouch! That second dataset is extremely uneven - over ten times as many non-Indian recipes as Indian ones!

The problem is: **this is usually how data looks in the real world.** You rarely have even numbers between your classes if you did not collect the data with this classification in mind, and you often think "more data is better data." We'll see how it plays out when we actually run our classifiers!

### Testing our datasets

We're going to use a `TfidfVectorizer` to convert ingredient lists to numbers, run a test/train split, and then train (and test) a `LinearSVC` classifier on the results. We'll start with the **balanced dataset**.

### Balanced dataset

In [ ]:
# Create a vectorizer and train it
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(df_balanced.ingredient_list)

# Features are our matrix of tf-idf values
# labels are whether each recipe is Indian or not
X = matrix
y = df_balanced.is_indian

# How many are Indian?
y.value_counts()

1    3000
0    3000
Name: is_indian, dtype: int64

We still have an even split, 3000 non-Indian recipes and 3000 Indian recipes. Let's run a test/train split and see how the results look.

In [ ]:
# Split into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Build a classifier and train it
clf = LinearSVC()
clf.fit(X_train, y_train)

# Test our classifier and build a confusion matrix
y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not indian', 'indian'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.966988,0.033012
Is indian,0.059508,0.940492


**Our classifier looks pretty good!** Around 96% accuracy for predicting non-Indian food, and around 95% correctly predicting Indian food. High quality *and* even.

Let's move on to see how it looks with our **unabalanced dataset**.

### Unbalanced dataset

In [ ]:
# Create a vectorizer and train it
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(df_unbalanced.ingredient_list)

# Features are our matrix of tf-idf values
# labels are whether each recipe is Indian or not
X = matrix
y = df_unbalanced.is_indian

# How many are Indian?
y.value_counts()

0    36771
1     3003
Name: is_indian, dtype: int64

Again: around 36k non-Indian recipes massively outweighing the 3,003 Indian recipes. While we love the world of big data, let's see what that imbalance does to our classifier.

In [ ]:
# Split our dataset is train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train the classifier on the training data
clf = LinearSVC()
clf.fit(X_train, y_train)

# Test our classifier and build a confusion matrix
y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not indian', 'indian'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.992817,0.007183
Is indian,0.168212,0.831788


Ouch!!! While we're doing **really well** at predicting non-Indian dishes, our ability to predict Indian dishes has plummeted to just over 80%.

Why? An easy way to think about it is **when it's a risky or rare decision, it's always safest to guess "not Indian."** In fact, if we *always guessed non-Indian*, no matter what, we'd be right...

In [ ]:
36771/(36771+3003)

0.9244984160506864

About 92% of the time! So how do we solve this problem?

### Solving the problem

Solving the problem of unbalanced (or biased) input classes is not too hard! There's a nice library that can give us a hand, [imbalanced-learn](https://imbalanced-learn.readthedocs.io/en/stable/).

imbalanced-learn will **resample** our dataset, either generating new datapoints or pruning out existing datapoints, until the classes are evened--providing a class-stratified sample.

#### What do we resample?

An important thing to note is that **bias occurs when we train our model.** If we show our model a skewed view of the world, it'll carry that bias when making judgments in the future. When we add or remove datapoints to even the problem, **we only need to do this for the training data.**

We want to show the model an even view of the world, so we give it even data. The test data should still reflect the "real" world. Before we were looking at how imbalanced our overall dataset was, but now let's **just look at how biased the training data is.**

In [ ]:
y_train.value_counts()

0    27582
1     2248
Name: is_indian, dtype: int64

In [ ]:
y_train.value_counts(normalize=True)

0    0.92464
1    0.07536
Name: is_indian, dtype: float64

Looks like a little over 7% of our training data is Indian - we'd like to get that up to 50%, so let's see what the imbalanced-learn library can do for us!

#### Undersampling

If we're feeling guilty that there are so many additional non-Indian recipes, *we could always get rid of those extra non-Indian recipes!* In fact, the balanced dataset manually created to form an even split of Indian/non-Indian recipes.

Instead of manually digging through our dataset to even things out, though, we can rely on imbalanced-learn to do it automatically. We'll use the technique of **undersampling** to take those ~28k non-Indian recipes and randomly filter them down to around 2,000 to match the number of Indian recipes. (Remember we're only doing this with training data!)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

resampler = RandomUnderSampler()
# Resample X and y so there are equal numbers of each y
X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)

y_train_resampled.value_counts()

0    2248
1    2248
Name: is_indian, dtype: int64

Awesome; equal numbers! Let's see how the classifier performs.

In [ ]:
# We already split our data, so we don't need to do that again

# Train the classifier on the resampled training data
clf = LinearSVC()
clf.fit(X_train_resampled, y_train_resampled)

# Build a confusion matrix
y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not indian', 'indian'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.96017,0.03983
Is indian,0.04106,0.95894


Looking good! It performs as well as our other 3,000/3,000 split because, well, it's more or less the same thing (although the test data is "realistically" unbalanced).

#### Oversampling

Cutting out those 27,000 "extra" non-Indian recipes seems like such a bummer, though. Wouldn't it be nice if we somehow found another 25,000 Indian recipes to even up our unbalanced training dataset to 27k non-Indian and 27k Indian? It's possible with **oversampling!**

Oversampling generates **new datapoints** based on your existing dataset. In this case we're going to use the `RandomOverSampler`, which just fills our dataset with **copies of the less-included class**. This is a form of data augmentation, described earlier when we generated additional image data through copying, perturbing and noising data. In this case, we'll have 27k Indian recipes, *but they'll be 25,0000 copies of the original ones*. Can that possibly help?

In [ ]:
from imblearn.over_sampling import RandomOverSampler

resampler = RandomOverSampler()
X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)

In [ ]:
y_train_resampled.value_counts()

0    27582
1    27582
Name: is_indian, dtype: int64

Looking good, a nice even 27,599 apiece. Let's see how the classifier works out!

In [ ]:
# We already split our dataset into train and test data

# Train the classifier on the resampled training data
clf = LinearSVC()
clf.fit(X_train_resampled, y_train_resampled)

# Build a confusion matrix with the result
y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not indian', 'indian'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.973120,0.026880
Is indian,0.064901,0.935099


Also looking pretty good! A little bit better at predicting non-Indian dishes and a little bit worse at predicting Indian dishes, but very simimlar to the undersampled example.

There are also other oversampling techniques that involve **creating synthetic data,** new datapoints that aren't *copies* of our data, but rather totally new ones. You can read more about them [on the imbalanced-learn page](https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html) and as reflected in the homework for our notebooks on data and regularization.

#### Review

In this section we talked about the problem of **imbalanced classes**, where an uneven split in your labels can cause suboptimal classifier performance. We used the imbalanced-learn library to talk about two methods of solving the issue - undersampling and oversampling - which both boosted performance as compared to the imbalanced dataset by stratifying data on the class of interest.

#### Discussion topics

What is the difference between oversampling and undersampling? Why might have oversampling done a better job predicting non-Indian recipes?

Why did we only resample the training data, and not the test data?

While the idea of automatically-generated fake data might sound more attractive than just re-using existing data, [what might be some issues with it](https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html)?

Can we think of any times when we might *not* want a balanced dataset?

You can find more examples with imbalanced datasets in the [example gallery](https://imbalanced-learn.org/stable/auto_examples/index.html#general-examples).

## Bootstrap sampling and sub-sampling

Bootstrap sampling and sub-sampling are two methods that are particularly relevant in the context of machine and deep learning for drawing inferences from our models.

[text drawn from this blog](https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/).

The bootstrap method is a resampling technique used to estimate statistics on a population by sampling a dataset **with replacement**. A bootstrap sample of a population excludes some data points (at random), and duplicates others.

Because bootstrap samples have the potential to exclude outliers, they can be used to estimate summary statistics such as the mean, standard deviation, or a confidence interval (e.g., 95%). It is used in applied machine learning to estimate the performance of machine learning models when making predictions on data not included in the training data, but may also to generate confidence intervals for "distances", "similarities", "probabilities" or other measurements drawn from auto-encoders or similar models (like [**this**](https://journals.sagepub.com/doi/full/10.1177/0003122419877135)). Confidence intervals cannot be directly inferred from other ML methods such as cross-validation.

### Bootstrap sampling example with word embeddings

In this section we explore the stability of word embeddings using bootstrap sampling. This is a well researched question in NLP research ([Evaluating the Stability of Embedding-based Word Similarities](https://mimno.infosci.cornell.edu/papers/antoniak-stability.pdf)). In the paper, *the authors come to the conclusion that there are several sources of variability
in cosine similarities between word embeddings vectors. The size of the corpus, the length of individual documents, and the presence or absence of specific documents can all affect the resulting embeddings. While differences in word association are measurable and are often significant, small differences in cosine similarity are not reliable, especially for small corpora. If the intention of a study is to learn about a specific corpus, we recommend practitioners test the statistical confidence of similarities based on word embeddings by training on multiple bootstrap samples.*

We will conduct a mini-version of this experiment to test how stable word similarities are after training with and without bootrstrap sampling on a smaller text dataset. We use a dataset we used before, the hobbies dataset. We note that bootstrapping can be performed with a deep auto-encoder or any deep model from which we seek to assess generalizable inferences.

In [ ]:
import gensim

In [ ]:
from yellowbrick.datasets import load_hobbies

In [ ]:
from gensim.parsing.preprocessing import preprocess_documents

In [ ]:
corpus = load_hobbies()

In [ ]:
preprocessed_texts = preprocess_documents(corpus.data)

In [ ]:
len(preprocessed_texts)

448

In [ ]:
from gensim.models import Word2Vec


In [ ]:
w2vmodel_cleaned = Word2Vec(
        preprocessed_texts,
        vector_size=100,
        window=10)

In [ ]:
w2vmodel_cleaned.wv.most_similar("book")

[('stori', 0.9999245405197144),
 ('approach', 0.9999086856842041),
 ('movi', 0.9999039173126221),
 ('show', 0.9999037981033325),
 ('reveal', 0.9999029636383057),
 ('live', 0.9999018311500549),
 ('left', 0.9999016523361206),
 ('illustr', 0.9999004602432251),
 ('todai', 0.9999003410339355),
 ('group', 0.9998995065689087)]

We use this as a reference while we explore how stable the model is with two sets of bootstrapped texts!

In [ ]:
from sklearn.utils import resample

In [ ]:
bootstrap_texts = resample(preprocessed_texts)

In [ ]:
len(bootstrap_texts)

448

In [ ]:
w2vmodel_boot = Word2Vec(
        bootstrap_texts,
        vector_size=100,
        window=10)

In [ ]:
w2vmodel_boot.wv.most_similar("book")

[('stori', 0.9996894001960754),
 ('author', 0.999585747718811),
 ('war', 0.9995577335357666),
 ('film', 0.9995225667953491),
 ('movi', 0.9995080828666687),
 ('publish', 0.9994944334030151),
 ('writer', 0.9994939565658569),
 ('novel', 0.9994746446609497),
 ('charact', 0.9994713664054871),
 ('compani', 0.9994663000106812)]

In [ ]:
bootstrap_texts_1 = resample(preprocessed_texts)

In [ ]:
w2vmodel_boot_1 = Word2Vec(
        bootstrap_texts_1,
        vector_size=100,
        window=10)

In [ ]:
w2vmodel_boot_1.wv.most_similar("book")

[('stori', 0.9998213648796082),
 ('novel', 0.9998008012771606),
 ('comic', 0.9997929930686951),
 ('author', 0.9997838735580444),
 ('write', 0.999782383441925),
 ('there’', 0.9997755289077759),
 ('writer', 0.999765932559967),
 ('audienc', 0.9997559785842896),
 ('movi', 0.9997525811195374),
 ('broken', 0.9997516870498657)]

Smaller corpora are likely to be more variable in their embeddings; when bootstrapped we see this more clearly. We encourage you to read the paper on stability in word embeddings and to run your own experiments with your datasets.

### Confidence intervals

How do we add a confidence interval to one of these cosine distances (e.g., the cosine similarity between "book" and "stori") with bootstrap sampling?

For example, if we assume the texts underlying our word embedding model are observations drawn from an independent and identically distributed (i.i.d.) population of cultural observations, then bootstrapping allows us to estimate the variance of word distances and projections by measuring those properties through sampling the empirical distribution of texts with replacement (Efron 2003; Efron and Tibshirani 1994).

To estimate bootstrapped 90 percent confidence intervals, the analyst draws documents with replacement from the corpus to construct 20 new corpora, each the size of the original corpus. The analyst then estimates either word similarities or angles between vectors on all 20 of these new corpora. The 2nd order (2nd smallest) estimated statistic $s(2)$ is taken as the confidence interval’s lower bound and the 19th order statistic $s(19)$ as its upper bound. The distance between $s(2)$ and $s(19)$ across 20 bootstrap samples span the 5th to the 95th percentiles of the statistic’s variance, bounding the 90th confidence interval. A 95 percent confidence interval would span $s(2)$ and $s(39)$ in word embedding distances or projections estimated on 40 bootstrap samples of a corpus, tracing the 2.5th to 97.5th percentiles.



In [ ]:
word_differences = []

In [ ]:
for i in range(0, 20):
  bootstrap_texts = resample(preprocessed_texts)
  w2vmodel_boot = Word2Vec(
        bootstrap_texts,
        vector_size=100,
        window=10)
  word_differences.append(w2vmodel_boot.similarity('book', 'stori'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


In [ ]:
sorted_diffs = sorted(word_differences)

In [ ]:
sorted_diffs

[0.9910327,
 0.9961562,
 0.99721795,
 0.99822146,
 0.99870205,
 0.9987051,
 0.9991782,
 0.9992429,
 0.99959224,
 0.99960154,
 0.99964875,
 0.99966174,
 0.9996634,
 0.9997253,
 0.99974334,
 0.99976045,
 0.9998034,
 0.99982756,
 0.99983954,
 0.9999032]

In [ ]:
sorted_diffs[18] - sorted_diffs[1]

0.0036833286

In [ ]:
(sorted_diffs[1], sorted_diffs[18])

(0.9961562, 0.99983954)

We have a 90% confidence interval that the difference between book and stori lies between (0.99588996, 0.999865).

### Sub-sampling

Unlike bootstrap sampling, in sub-sampling we sample without replacement, usually in cases when we are dealing with very large data and only want a portion of it. [subsample](https://pypi.org/project/subsample/) is a python package that works as a command-line tool for sub-sampling, and is especially powerful with text based datasets. It includes methods such as reservoir sampling and two pass sampling.

Subsampling also refers to the process of randomly partitioning your dataset and running the same algorithm (e.g., your deep model) on each subsample, then using these to estimate statistics like the mean, standard deviation or confidence intervals. It is especially useful when the data is large, model optimization is slow, and running the same model on smaller data yields superlinear speed increases (as with [**this case**](https://journals.sagepub.com/doi/full/10.1177/0003122419877135)) of text auto-encoders.)

By randomly partitioning the data (e.g., a corpus of texts) into non-overlapping samples, estimates of neural network models on these subsets allow calculation of confidence or credible intervals as a function of the empirical distribution of distance or projection statistics and number of texts in the subsample (Politis, Romano, and Wolf 1997). Subsampling relies on the same i.i.d. assumption as the bootstrap (Politis and Romano 1992, 1994). For 90 percent confidence intervals, we randomly partition the corpus into 20 subcorpora, then calculate the error of our embedding distance or projection statistic s for each subsample $k$ as $B^k=\sqrt{\tau^k}(s^k−\bar{s})$, where $\tau^k$ is the number of texts in subsample $k$, $s^k$ is the embedding distance or projection for the $k$th sample, and $\bar{s}$ is the mean of the 20 estimates. The 90 percent confidence interval spans the 5th to 95th percentile variances, inscribed by $\bar{s}-\frac{B^k(19)}{\sqrt{\tau}}$ and $\bar{s}-\frac{B^k(2)}{\sqrt{\tau^k}}$ where $\tau$ is the number of texts in the total corpus. As with bootstrapping, a 95 percent confidence interval would require 40 subsamples; a 99 percent confidence would require 200 (.5th to 99.5th percentiles).

NOTE: since this dataset is really small, it doesn't really make sense to do sub-sampling; but for the sake of demonstrating it we will have code below for it.

In [ ]:
np.random.seed(42)

In [ ]:
np.random.shuffle(preprocessed_texts)

In [ ]:
len(preprocessed_texts) / 23

19.47826086956522

In [ ]:
chunks = [preprocessed_texts[x:x+23] for x in range(0, len(preprocessed_texts), 23)]

In [ ]:
chunks[0][0][0:10]

['kevork',
 'djansezian',
 'getti',
 'imag',
 'kobe',
 'bryant',
 'hadn',
 'top',
 'point',
 'game']

In [ ]:
len(chunks)

20

In [ ]:
word_differences = []

In [ ]:
for i in range(0, 20):
  sub_sampled = chunks[i]
  w2vmodel_sub = Word2Vec(
        sub_sampled,
        vector_size=100,
        window=10)
  try:
    word_differences.append(w2vmodel_sub.similarity('stori', 'book'))
  except:
    # we wouldn't want to do this normally!!
    # it's only because sometimes those two words don't appear in the sub-sample
    word_differences.append(np.random.uniform((0.3, 1))[0])
    print("Appended random value")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


Appended random value
Appended random value
Appended random value
Appended random value


In [ ]:
len(word_differences)

20

In [ ]:
word_differences

[0.95981264,
 0.96406794,
 0.82241315,
 0.41246882,
 0.7836734,
 0.7599779299501168,
 0.28337348,
 0.811027521593273,
 0.9652969,
 0.3153138,
 0.981836,
 0.6962260473458534,
 0.99631363,
 0.3317162,
 0.7111304,
 0.9446672,
 0.24160936,
 0.89898944,
 0.38395357,
 0.3841152534639495]

**NOTE** We replace the samples where there is no book or stori with a random value between 0.3 and 1 - this is only because we have a very small dataset, with larger datasets we would expect there to be a statistic or value for each sample.

In [ ]:
import numpy as np

In [ ]:
mean_difference = np.mean(word_differences)

In [ ]:
mean_difference

0.6823991362903357

In [ ]:
Bs = []

In [ ]:
for i in range(0, len(chunks)):
  B = np.sqrt(len(chunks)) * (word_differences[i] - mean_difference)
  Bs.append(B)

In [ ]:
interval_19 = mean_difference - ((Bs[18] * word_differences[18]) / len(chunks[18]))

In [ ]:
interval_2 = mean_difference - ((Bs[1] * word_differences[1]) / len(chunks[1]))

In [ ]:
(interval_2, interval_19)

(0.6295991823506403, 0.7046799477061303)

## Assignment Exercise

**1)** Run 3 probabilistic sampling methods and 2 non-probabilistic methods and explore the samples returned. How would sampling help with your data?

In [ ]:
sampling_help = 'value' #@param {type:"string"}

**2)** Find an imbalanced dataset and build a classifier to predict the label causing the imbalance. Explore undersampling and oversampling solutions to your dataset.

**3)** Use bootstrap sampling to test the stability of a word, sentence, or graph embedding.

# Module 2: Fine-Tuning with LoRA and QLoRA

**Summary:** Fine-tuning neural networks involves taking a pre-trained model and adjusting its parameters allowing it to perform well on a specific task or dataset. This process sometimes involves (1) retraining all model weights, (2) unfreezing selected layers of the network and training them with a low learning rate on the new data, while keeping other layers frozen to preserve useful features learned during initial training, or (3) building adapters that translate the model weights through low-rank approaches, the purpose of this section. Fine-tuning is particularly valuable when working with limited data or computational resources, as it leverages knowledge transferred from the pre-trained model while allowing adaptation to the target task.

**Readings:**
- Hu et al. (2021) "LoRA: Low-Rank Adaptation of Large Language Models"
- Dettmers et al. (2023) "QLoRA: Efficient Finetuning of Quantized LLMs"
- Rafailov et al. (2023) "Direct Preference Optimization: Your Language Model is Secretly a Reward Model"

# Install & Imports
First, we install any relevant libraries (if not already installed). This code will:
1. Install Hugging Face Transformers (for LLM loading & training).
2. Install bitsandbytes (if using QLoRA or 4-bit quantization).
3. Install datasets or any other library you need.

In [ ]:
!pip install transformers accelerate datasets bitsandbytes peft -q

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)

import os
import numpy as np
import time

# For demonstration, let's just pick a small-ish model to test
MODEL_NAME = "facebook/galactica-125m"  # or "EleutherAI/gpt-neo-125M"
print(f"Using model: {MODEL_NAME}")

# Background: LoRA & QLoRA

# **LoRA (Low-Rank Adaptation)**:
- Paper: _Hu et al. (2022) “LoRA: Low-Rank Adaptation of Large Language Models.” ICLR._
- Key idea: Freeze the original model weights, inject trainable low-rank matrices (the "adapters") into attention or MLP layers. Only the adapter weights are updated. This drastically reduces parameter count for fine-tuning, so we can fit training on a single GPU or smaller hardware.

# **QLoRA**:
- Paper: _Dettmers et al. (2023) “QLoRA: Efficient Finetuning of Quantized LLMs.”_
- Key idea: Quantize model weights (e.g., 4-bit) for forward/backward pass, and then add LoRA adapters. The base model remains in quantized form (saving memory), while the LoRA adapters (in higher precision) get trained.

LoRA typically wraps attention projection matrices, which will detail in future weeks (LLM parameters `W_q`, `W_k`, `W_v`, or `W_out`) with low-rank decomposition. QLoRA applies 4-bit quantization on those same weight matrices, but leaves LoRA adapters in FP16 or BF16.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Important for causal LM tasks; we want to pad on left if doing batch inference, etc.
tokenizer.pad_token = tokenizer.eos_token

# If we do standard full-precision load:
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  # or just "cpu" or "cuda:0" depending on environment
    torch_dtype=torch.float16
)


# If we want to do a 4-bit load using bitsandbytes:
# base_model_4bit = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     load_in_4bit=True,
#     device_map="auto",
#     quantization_config=bnb.QuantizationConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.float16,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type='nf4'
#     )
# )

print("Base model loaded!")

# (Optional) Inject LoRA Adapters
We can wrap the above model with [PEFT (Parameter-Efficient Fine-Tuning)](https://github.com/huggingface/peft) or with the original LoRA code.

Below is a **PEFT**-style example (huggingface/peft) for simplicity. If you want the original LoRA or QLoRA repos, see their `train.py` scripts. The logic is mostly the same.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["k_proj", "q_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model_lora = get_peft_model(base_model, lora_config)
print(f"LoRA Model params: {model_lora.print_trainable_parameters()}")

# **QLoRA**:
If using QLoRA with bitsandbytes 4-bit quantization, we could do something like:

In [ ]:
from peft import LoraConfig, get_peft_model

# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     target_modules=["query_key_value"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM
# )

# # base_model_4bit is the quantized model
# model_lora_4bit = get_peft_model(base_model_4bit, lora_config)
# print(model_lora_4bit.print_trainable_parameters())

# Then proceed with training, but the base weights remain 4-bit in memory, and only the LoRA adapter is stored in full precision.


# Prepare a Dataset
For a small demonstration, let's create a toy text dataset or use something from Hugging Face `datasets`.

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")  # small example
print(raw_dataset)

# Let's define a data collator for causal LM
from transformers import DataCollatorForLanguageModeling

def tokenize_function(example):
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model_lora.resize_token_embeddings(len(tokenizer))
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # for causal LM
)

train_dataset = tokenized_dataset["train"]
val_dataset   = tokenized_dataset["validation"]
test_dataset  = tokenized_dataset["test"]

# Fine-Tuning with LoRA or QLoRA

We'll use the Hugging Face `Trainer` to fine-tune.
# Training Arguments
Key parameters to watch for:
- `learning_rate` — might be smaller for LLM fine-tuning (e.g., 1e-4, 2e-5).
- `per_device_train_batch_size`, `per_device_eval_batch_size`.
- `gradient_accumulation_steps` (especially for bigger LLMs).
- `max_steps` or `num_train_epochs`.
- `fp16` or `bf16` if available (and stable).

If QLoRA, ensure `bitsandbytes` is installed and your GPU supports 4-bit.

In [ ]:
training_args = TrainingArguments(
    output_dir="lora-finetune-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_steps=50,
    fp16=True,            # Enable mixed precision training if possible
    report_to="none",     # or "wandb", "tensorboard", etc.
    max_grad_norm=1.0 # Clip gradients to prevent exploding gradients
)

# Set up the Trainer
trainer = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

In [ ]:
# Train!

trainer.train()

print("Done Training LoRA model.")

# Evaluate & Compare
Let's do a quick perplexity check on the validation (or test) set. We'll reuse the Trainer’s `evaluate()` method, which will compute the causal LM loss. Then perplexity = `exp(loss)`. If the perplexity is high the data is improbable; if the perplexity is low, then the data is expected and our models works well.


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)
val_loss = eval_results["eval_loss"]
val_ppl = np.exp(val_loss)
print(f"Validation Perplexity: {val_ppl:.2f}")

# Homework Assignments

1. **Adapt an LLM** (e.g., Galactical used above) with LoRA to your text and evaluate its perplexity before and after adaptation.

2. **QLoRA**: Load a 4-bit quantized model (via bitsandbytes), apply LoRA, and fine-tune. Compare GPU memory usage, speed, and resulting perplexity with the standard LoRA approach above.

3. **Hyperparameter Tuning**: Adjust `r`, `lora_alpha`, `lora_dropout`, or the learning rate.

4. **Regularization**: Introduce weight decay, dropout in LoRA layers, or regularization of adapter weights.

# Module 3: Benchmarking LLM Agents

Evaluating LLM-based agents requires systematic benchmarking approaches that go beyond traditional metrics. This module covers:

1. **Centered Kernel Alignment (CKA)** - A metric for comparing neural network representations
2. **Agent Benchmarking Frameworks** - Systematic evaluation of agent capabilities
3. **Bias in Benchmarks** - Understanding how benchmark design affects evaluation

**Key Readings:**
- Kornblith et al. (2019) "Similarity of Neural Network Representations Revisited" (CKA)
- Mohammadi et al. (2025) "Evaluation and Benchmarking of LLM Agents: A Survey"
- Kapoor et al. (2024) "AI Agents That Matter"
- Liu et al. (2024) "AgentBench: Evaluating LLMs as Agents"

## Part 1: Centered Kernel Alignment (CKA)

CKA is a method for comparing representations learned by different neural networks or different layers within the same network. It measures similarity in a way that is:
- **Invariant to orthogonal transformations** (e.g., neuron permutation)
- **Invariant to isotropic scaling**

This makes it particularly useful for understanding:
- How representations evolve during training
- How different architectures learn similar or different representations
- Which layers are most important for specific tasks

### Mathematical Foundation

The CKA score is defined as:

$$\texttt{CKA}(\mathbf{K}, \mathbf{L}) = \frac{\texttt{HSIC}(\mathbf{K}, \mathbf{L})}{\sqrt{\texttt{HSIC}(\mathbf{K}, \mathbf{K})\texttt{HSIC}(\mathbf{L}, \mathbf{L})}}$$

where HSIC is the Hilbert-Schmidt Independence Criterion:

$$\texttt{HSIC}(\mathbf{K}, \mathbf{L}) = \frac{\text{tr}(\mathbf{K} \mathbf{H}_m \mathbf{L} \mathbf{H}_m)}{(m-1)^2}$$

with $\mathbf{H}_m = \mathbf{I}_m - \frac{1}{m} \mathbf{1}\mathbf{1}^T$.

In [ ]:
# Install dependencies
!pip install torch numpy matplotlib seaborn -q

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple

In [ ]:
def linear_CKA(X: np.ndarray, Y: np.ndarray) -> float:
    """
    Compute Linear Centered Kernel Alignment between two sets of representations.
    
    Args:
        X: (n_samples, n_features_1) - representations from first source
        Y: (n_samples, n_features_2) - representations from second source
    
    Returns:
        CKA similarity score between 0 and 1
    """
    # Center the data
    X = X - X.mean(axis=0)
    Y = Y - Y.mean(axis=0)
    
    # Compute Gram matrices (linear kernel)
    K = X @ X.T
    L = Y @ Y.T
    
    # Compute HSIC
    def hsic(K, L):
        n = K.shape[0]
        H = np.eye(n) - np.ones((n, n)) / n
        return np.trace(K @ H @ L @ H) / (n - 1) ** 2
    
    # Compute CKA
    hsic_kl = hsic(K, L)
    hsic_kk = hsic(K, K)
    hsic_ll = hsic(L, L)
    
    return hsic_kl / np.sqrt(hsic_kk * hsic_ll)


def rbf_CKA(X: np.ndarray, Y: np.ndarray, sigma: float = 1.0) -> float:
    """
    Compute RBF (Radial Basis Function) CKA between two sets of representations.
    
    Args:
        X: (n_samples, n_features_1) - representations from first source
        Y: (n_samples, n_features_2) - representations from second source
        sigma: RBF kernel bandwidth
    
    Returns:
        CKA similarity score between 0 and 1
    """
    def rbf_kernel(X, sigma):
        # Compute pairwise squared Euclidean distances
        sq_dists = np.sum(X**2, axis=1, keepdims=True) + np.sum(X**2, axis=1) - 2 * X @ X.T
        return np.exp(-sq_dists / (2 * sigma**2))
    
    K = rbf_kernel(X, sigma)
    L = rbf_kernel(Y, sigma)
    
    def hsic(K, L):
        n = K.shape[0]
        H = np.eye(n) - np.ones((n, n)) / n
        return np.trace(K @ H @ L @ H) / (n - 1) ** 2
    
    hsic_kl = hsic(K, L)
    hsic_kk = hsic(K, K)
    hsic_ll = hsic(L, L)
    
    return hsic_kl / np.sqrt(hsic_kk * hsic_ll)

### Demonstration: Comparing Neural Network Layer Representations

Let's create a simple example to demonstrate CKA by comparing representations from different layers of a neural network.

In [ ]:
# Define a simple neural network with hooks to capture layer activations
class SimpleNet(nn.Module):
    def __init__(self, input_dim=784, hidden_dims=[256, 128, 64], output_dim=10):
        super().__init__()
        self.layers = nn.ModuleList()
        dims = [input_dim] + hidden_dims
        for i in range(len(dims) - 1):
            self.layers.append(nn.Linear(dims[i], dims[i+1]))
            self.layers.append(nn.ReLU())
        self.output = nn.Linear(hidden_dims[-1], output_dim)
        
        # Storage for layer activations
        self.activations = []
        
    def forward(self, x):
        self.activations = [x.detach().numpy()]  # Store input
        for layer in self.layers:
            x = layer(x)
            if isinstance(layer, nn.ReLU):
                self.activations.append(x.detach().numpy())
        x = self.output(x)
        self.activations.append(x.detach().numpy())
        return x

# Create two networks (simulating different random initializations)
torch.manual_seed(42)
net1 = SimpleNet()
torch.manual_seed(123)
net2 = SimpleNet()

# Generate random input data
X = torch.randn(100, 784)

# Get activations from both networks
with torch.no_grad():
    _ = net1(X)
    activations1 = net1.activations.copy()
    _ = net2(X)
    activations2 = net2.activations.copy()

print(f"Number of layers captured: {len(activations1)}")
for i, act in enumerate(activations1):
    print(f"Layer {i}: shape {act.shape}")

In [ ]:
def compute_cka_matrix(activations1: List[np.ndarray], 
                       activations2: List[np.ndarray]) -> np.ndarray:
    """
    Compute pairwise CKA scores between all layers of two networks.
    """
    n1, n2 = len(activations1), len(activations2)
    cka_matrix = np.zeros((n1, n2))
    
    for i in range(n1):
        for j in range(n2):
            cka_matrix[i, j] = linear_CKA(activations1[i], activations2[j])
    
    return cka_matrix


def plot_cka_matrix(cka_matrix: np.ndarray, 
                    title: str = "CKA Similarity Matrix",
                    xlabel: str = "Network 2 Layers",
                    ylabel: str = "Network 1 Layers"):
    """
    Plot a CKA similarity matrix as a heatmap.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(cka_matrix, cmap='viridis', vmin=0, vmax=1)
    
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_title(title, fontsize=14)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('CKA Score', fontsize=12)
    
    # Add layer labels
    n1, n2 = cka_matrix.shape
    ax.set_xticks(range(n2))
    ax.set_yticks(range(n1))
    ax.set_xticklabels([f'L{i}' for i in range(n2)])
    ax.set_yticklabels([f'L{i}' for i in range(n1)])
    
    plt.tight_layout()
    return fig, ax


# Compute and plot CKA matrix for self-similarity
cka_self = compute_cka_matrix(activations1, activations1)
plot_cka_matrix(cka_self, title="Self-Similarity (Same Network)", 
                xlabel="Layer", ylabel="Layer")
plt.show()

# Compute and plot CKA matrix between two networks
cka_cross = compute_cka_matrix(activations1, activations2)
plot_cka_matrix(cka_cross, title="Cross-Network Similarity (Different Seeds)")
plt.show()

## Part 2: Benchmarking LLM Agents

Evaluating LLM-based agents goes beyond simple accuracy metrics. According to Mohammadi et al. (2025) and Kapoor et al. (2024), comprehensive agent evaluation should consider:

### Key Evaluation Dimensions

1. **Task Completion Rate** - Can the agent complete the assigned task?
2. **Efficiency** - How many steps/tokens/API calls does it take?
3. **Safety** - Does the agent avoid harmful actions?
4. **Robustness** - How well does the agent handle edge cases?
5. **Generalization** - Does performance transfer to new domains?

### Common Benchmarks

| Benchmark | Focus Area | Key Metrics |
|-----------|------------|-------------|
| AgentBench | General agent capabilities | Success rate, efficiency |
| WebArena | Web navigation | Task completion, # actions |
| SWE-Bench | Software engineering | Resolved issues % |
| ToolBench | Tool use | API call accuracy |

In [ ]:
# Simple Agent Evaluation Framework

from dataclasses import dataclass
from typing import Callable, Dict, Any, Optional
import time

@dataclass
class BenchmarkTask:
    """A single benchmark task for agent evaluation."""
    task_id: str
    description: str
    input_data: Any
    expected_output: Any
    max_steps: int = 10
    timeout_seconds: float = 60.0

@dataclass 
class EvaluationResult:
    """Results from evaluating an agent on a task."""
    task_id: str
    success: bool
    steps_taken: int
    time_elapsed: float
    agent_output: Any
    error_message: Optional[str] = None


class AgentBenchmark:
    """Framework for benchmarking LLM agents."""
    
    def __init__(self, name: str):
        self.name = name
        self.tasks: List[BenchmarkTask] = []
        self.results: List[EvaluationResult] = []
    
    def add_task(self, task: BenchmarkTask):
        """Add a task to the benchmark."""
        self.tasks.append(task)
    
    def evaluate_agent(self, agent_fn: Callable, 
                       evaluator_fn: Callable[[Any, Any], bool]) -> Dict[str, float]:
        """
        Evaluate an agent on all benchmark tasks.
        
        Args:
            agent_fn: Function that takes (input_data) and returns (output, steps)
            evaluator_fn: Function that takes (output, expected) and returns bool
        
        Returns:
            Dictionary of evaluation metrics
        """
        self.results = []
        
        for task in self.tasks:
            start_time = time.time()
            try:
                output, steps = agent_fn(task.input_data)
                elapsed = time.time() - start_time
                success = evaluator_fn(output, task.expected_output)
                
                result = EvaluationResult(
                    task_id=task.task_id,
                    success=success,
                    steps_taken=steps,
                    time_elapsed=elapsed,
                    agent_output=output
                )
            except Exception as e:
                result = EvaluationResult(
                    task_id=task.task_id,
                    success=False,
                    steps_taken=0,
                    time_elapsed=time.time() - start_time,
                    agent_output=None,
                    error_message=str(e)
                )
            
            self.results.append(result)
        
        # Compute aggregate metrics
        success_rate = sum(r.success for r in self.results) / len(self.results)
        avg_steps = np.mean([r.steps_taken for r in self.results if r.success])
        avg_time = np.mean([r.time_elapsed for r in self.results])
        
        return {
            'success_rate': success_rate,
            'avg_steps_on_success': avg_steps if not np.isnan(avg_steps) else 0,
            'avg_time_seconds': avg_time,
            'total_tasks': len(self.tasks)
        }

print("Agent Benchmark Framework loaded!")

In [ ]:
# Example: Simple Math Agent Benchmark

# Create a simple benchmark
math_benchmark = AgentBenchmark("Simple Math")

# Add tasks
math_tasks = [
    BenchmarkTask("add_1", "Add two numbers", (3, 5), 8),
    BenchmarkTask("add_2", "Add two numbers", (10, 20), 30),
    BenchmarkTask("multiply_1", "Multiply two numbers", (4, 7), 28),
    BenchmarkTask("complex_1", "Complex calculation", (2, 3, 4), 14),  # 2*3 + 4 + 4
]

for task in math_tasks:
    math_benchmark.add_task(task)

# Define a simple agent
def simple_math_agent(input_data):
    """A simple agent that performs math operations."""
    steps = 1
    if len(input_data) == 2:
        # Try addition first, then multiplication
        result = input_data[0] + input_data[1]
        steps = 1
    elif len(input_data) == 3:
        result = input_data[0] * input_data[1] + input_data[2] + input_data[2]
        steps = 3
    else:
        result = sum(input_data)
        steps = len(input_data)
    return result, steps

# Define evaluator
def exact_match_evaluator(output, expected):
    return output == expected

# Run evaluation
results = math_benchmark.evaluate_agent(simple_math_agent, exact_match_evaluator)

print("\n=== Benchmark Results ===")
for key, value in results.items():
    print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")

print("\n=== Per-Task Results ===")
for result in math_benchmark.results:
    status = "✓" if result.success else "✗"
    print(f"{status} {result.task_id}: output={result.agent_output}, steps={result.steps_taken}")

## Part 3: Bias in Benchmark Design

Benchmark design can introduce systematic biases that affect how we evaluate AI agents:

### Sources of Bias

1. **Selection Bias** - Tasks may not represent real-world distribution
2. **Cultural Bias** - Benchmarks often reflect Western, English-centric perspectives
3. **Temporal Bias** - Training data contamination (models may have seen test data)
4. **Difficulty Calibration** - Task difficulty may not scale appropriately

### Mitigation Strategies

- Use held-out test sets with temporal splits
- Include diverse cultural and linguistic examples
- Report confidence intervals, not just point estimates
- Use multiple benchmarks to avoid overfitting to one metric

### Model Collapse Warning

As noted in Shumailov et al. (2024), models trained on recursively generated data can collapse. This has implications for benchmark creation:
- Synthetic benchmarks may not capture real-world complexity
- Over-reliance on LLM-generated evaluation data can be problematic

In [ ]:
# Demonstration: How benchmark composition affects reported performance

import numpy as np
import matplotlib.pyplot as plt

# Simulate an agent with different performance on different task types
np.random.seed(42)

# Agent performance on different task categories (success probability)
agent_performance = {
    'english_text': 0.90,
    'multilingual': 0.60,
    'math': 0.75,
    'code': 0.80,
    'reasoning': 0.70
}

def simulate_benchmark(task_distribution: Dict[str, int], 
                       performance: Dict[str, float], 
                       n_runs: int = 100) -> Tuple[float, float]:
    """
    Simulate benchmark results given task distribution and agent performance.
    
    Returns: (mean_success_rate, std_success_rate)
    """
    all_rates = []
    
    for _ in range(n_runs):
        successes = 0
        total = 0
        
        for task_type, count in task_distribution.items():
            prob = performance.get(task_type, 0.5)
            successes += np.random.binomial(count, prob)
            total += count
        
        all_rates.append(successes / total)
    
    return np.mean(all_rates), np.std(all_rates)

# Define different benchmark compositions
benchmarks = {
    'English-Heavy': {'english_text': 80, 'multilingual': 5, 'math': 5, 'code': 5, 'reasoning': 5},
    'Balanced': {'english_text': 20, 'multilingual': 20, 'math': 20, 'code': 20, 'reasoning': 20},
    'Technical': {'english_text': 10, 'multilingual': 10, 'math': 30, 'code': 30, 'reasoning': 20},
    'Multilingual': {'english_text': 20, 'multilingual': 60, 'math': 10, 'code': 5, 'reasoning': 5}
}

# Simulate and plot results
results = {}
for name, dist in benchmarks.items():
    mean, std = simulate_benchmark(dist, agent_performance)
    results[name] = (mean, std)

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
names = list(results.keys())
means = [results[n][0] for n in names]
stds = [results[n][1] for n in names]

bars = ax.bar(names, means, yerr=stds, capsize=5, color=['#2ecc71', '#3498db', '#9b59b6', '#e74c3c'])
ax.set_ylabel('Success Rate', fontsize=12)
ax.set_xlabel('Benchmark Type', fontsize=12)
ax.set_title('Same Agent, Different Benchmark Compositions', fontsize=14)
ax.set_ylim(0, 1)
ax.axhline(y=0.75, color='gray', linestyle='--', label='"True" average performance')
ax.legend()

for bar, mean in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03, 
            f'{mean:.1%}', ha='center', fontsize=11)

plt.tight_layout()
plt.show()

print("\nKey insight: The SAME agent shows very different 'performance' ")
print("depending on how the benchmark tasks are composed!")

## Module 3 Exercises

### Exercise 1: CKA Analysis
Train a simple neural network on MNIST or CIFAR-10, save checkpoints during training, and use CKA to analyze how representations change during training.

### Exercise 2: Agent Benchmark Design
Design a benchmark for a specific domain (e.g., question answering, code generation). Consider:
- What tasks should be included?
- What metrics matter most?
- How would you ensure the benchmark is fair and unbiased?

### Exercise 3: Critical Analysis
Read Kapoor et al. (2024) "AI Agents That Matter" and write a 1-page summary of:
- What makes a benchmark "good"?
- What are common pitfalls in agent evaluation?
- How would you apply these lessons to your research?

# Module 4: Tools and the Model Context Protocol (MCP)

Modern LLM agents extend their capabilities by interacting with external tools, APIs, and data sources. This module covers:

1. **Tool Use Patterns** - How LLMs interact with external functions
2. **Model Context Protocol (MCP)** - A standardized protocol for providing context to AI models
3. **Building Tool-Using Agents** - Practical implementation

**Key Readings:**
- Anthropic (2024) "What is the Model Context Protocol (MCP)?"
- Epperson et al. (2025) "Interactive Debugging and Steering of Multi-Agent AI Systems"
- Khattab et al. (2023) "DSPy: Compiling Declarative Language Model Calls into Self-Improving Pipelines"

## Part 1: Understanding Tool Use in LLMs

Tool use allows LLMs to:
- **Access real-time information** (web search, databases)
- **Perform calculations** (code execution, math)
- **Take actions** (send emails, create files)
- **Interact with external services** (APIs, cloud services)

### Tool Use Architecture

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   User      │────▶│   LLM       │────▶│   Tool      │
│   Request   │     │   Agent     │     │   Executor  │
└─────────────┘     └─────────────┘     └─────────────┘
                           │                   │
                           ▼                   ▼
                    ┌─────────────┐     ┌─────────────┐
                    │   Tool      │     │   External  │
                    │   Selection │     │   Service   │
                    └─────────────┘     └─────────────┘
```

### Key Concepts

1. **Tool Definition** - JSON schema describing the tool's interface
2. **Tool Selection** - LLM decides which tool(s) to use
3. **Parameter Extraction** - LLM extracts arguments from user request
4. **Result Integration** - Tool output is fed back to the LLM

In [ ]:
# Install dependencies
!pip install openai anthropic requests -q

In [ ]:
import json
from typing import Callable, Dict, Any, List
from dataclasses import dataclass, field
import inspect

@dataclass
class Tool:
    """Represents a tool that an LLM can use."""
    name: str
    description: str
    parameters: Dict[str, Any]
    function: Callable
    
    def to_schema(self) -> Dict[str, Any]:
        """Convert to OpenAI/Anthropic tool schema format."""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters
            }
        }
    
    def execute(self, **kwargs) -> Any:
        """Execute the tool with given arguments."""
        return self.function(**kwargs)


class ToolRegistry:
    """Registry for managing available tools."""
    
    def __init__(self):
        self.tools: Dict[str, Tool] = {}
    
    def register(self, tool: Tool):
        """Register a new tool."""
        self.tools[tool.name] = tool
    
    def get(self, name: str) -> Tool:
        """Get a tool by name."""
        return self.tools.get(name)
    
    def list_schemas(self) -> List[Dict]:
        """Get schemas for all registered tools."""
        return [tool.to_schema() for tool in self.tools.values()]
    
    def execute(self, name: str, **kwargs) -> Any:
        """Execute a tool by name."""
        tool = self.get(name)
        if tool is None:
            raise ValueError(f"Tool '{name}' not found")
        return tool.execute(**kwargs)

print("Tool framework loaded!")

In [ ]:
# Define example tools

def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        # Simple and safe evaluation (for demo only)
        allowed_chars = set('0123456789+-*/().^ ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression.replace('^', '**'))
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

def get_weather(city: str) -> str:
    """Get current weather for a city (simulated)."""
    # Simulated weather data
    import random
    random.seed(hash(city) % 100)
    temp = random.randint(40, 90)
    conditions = random.choice(['Sunny', 'Cloudy', 'Rainy', 'Partly Cloudy'])
    return f"Weather in {city}: {temp}°F, {conditions}"

def search_database(query: str, limit: int = 5) -> str:
    """Search a database (simulated)."""
    # Simulated database results
    results = [
        f"Result {i+1}: Document about '{query}' - relevance: {0.9 - i*0.1:.1f}"
        for i in range(limit)
    ]
    return "\n".join(results)

# Create tool registry and register tools
registry = ToolRegistry()

registry.register(Tool(
    name="calculator",
    description="Evaluate mathematical expressions. Use for any calculations.",
    parameters={
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "Mathematical expression to evaluate (e.g., '2 + 2', '3 * 4')"
            }
        },
        "required": ["expression"]
    },
    function=calculator
))

registry.register(Tool(
    name="get_weather",
    description="Get the current weather for a specified city.",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city (e.g., 'New York', 'London')"
            }
        },
        "required": ["city"]
    },
    function=get_weather
))

registry.register(Tool(
    name="search_database",
    description="Search a database for relevant documents.",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query"
            },
            "limit": {
                "type": "integer",
                "description": "Maximum number of results (default: 5)",
                "default": 5
            }
        },
        "required": ["query"]
    },
    function=search_database
))

# Display registered tools
print("Registered Tools:")
print(json.dumps(registry.list_schemas(), indent=2))

In [ ]:
# Demonstrate tool execution

print("=== Calculator Tool ===")
print(registry.execute("calculator", expression="2 + 2 * 3"))
print(registry.execute("calculator", expression="(10 + 5) ^ 2"))

print("\n=== Weather Tool ===")
print(registry.execute("get_weather", city="Chicago"))
print(registry.execute("get_weather", city="San Francisco"))

print("\n=== Database Search Tool ===")
print(registry.execute("search_database", query="machine learning", limit=3))

## Part 2: Model Context Protocol (MCP)

The **Model Context Protocol (MCP)** is an open standard developed by Anthropic that enables AI models to securely access data from various sources. MCP provides a standardized way for LLMs to:

1. **Access local files and databases**
2. **Query external APIs and services**
3. **Interact with development tools**
4. **Maintain context across sessions**

### MCP Architecture

```
┌─────────────────────────────────────────────────────┐
│                    MCP Host                         │
│  (Claude Desktop, IDE, or Custom Application)      │
└───────────────────────┬─────────────────────────────┘
                        │ MCP Protocol
        ┌───────────────┼───────────────┐
        ▼               ▼               ▼
┌──────────────┐ ┌──────────────┐ ┌──────────────┐
│  MCP Server  │ │  MCP Server  │ │  MCP Server  │
│  (Files)     │ │  (Database)  │ │  (API)       │
└──────────────┘ └──────────────┘ └──────────────┘
        │               │               │
        ▼               ▼               ▼
┌──────────────┐ ┌──────────────┐ ┌──────────────┐
│ Local Files  │ │  PostgreSQL  │ │  REST API   │
└──────────────┘ └──────────────┘ └──────────────┘
```

### Key Components

1. **MCP Host** - The application hosting the AI model (e.g., Claude Desktop)
2. **MCP Server** - Provides access to a specific data source or capability
3. **MCP Protocol** - JSON-RPC based communication protocol

### Server Capabilities

MCP servers can expose:
- **Resources** - File-like data (documents, images, etc.)
- **Tools** - Functions the AI can call
- **Prompts** - Template prompts for specific tasks

In [ ]:
# Simple MCP Server Implementation (Conceptual)

import json
from typing import Dict, Any, List, Optional
from dataclasses import dataclass, field
from abc import ABC, abstractmethod

@dataclass
class MCPResource:
    """Represents a resource exposed by an MCP server."""
    uri: str
    name: str
    description: str
    mime_type: str = "text/plain"

@dataclass
class MCPTool:
    """Represents a tool exposed by an MCP server."""
    name: str
    description: str
    input_schema: Dict[str, Any]


class MCPServer(ABC):
    """Base class for MCP servers."""
    
    def __init__(self, name: str, version: str = "1.0.0"):
        self.name = name
        self.version = version
        self._resources: Dict[str, MCPResource] = {}
        self._tools: Dict[str, MCPTool] = {}
    
    def get_server_info(self) -> Dict[str, Any]:
        """Return server information."""
        return {
            "name": self.name,
            "version": self.version,
            "protocolVersion": "2024-11-05"
        }
    
    def list_resources(self) -> List[Dict[str, Any]]:
        """List available resources."""
        return [
            {
                "uri": r.uri,
                "name": r.name,
                "description": r.description,
                "mimeType": r.mime_type
            }
            for r in self._resources.values()
        ]
    
    def list_tools(self) -> List[Dict[str, Any]]:
        """List available tools."""
        return [
            {
                "name": t.name,
                "description": t.description,
                "inputSchema": t.input_schema
            }
            for t in self._tools.values()
        ]
    
    @abstractmethod
    def read_resource(self, uri: str) -> str:
        """Read a resource by URI."""
        pass
    
    @abstractmethod
    def call_tool(self, name: str, arguments: Dict[str, Any]) -> Any:
        """Call a tool with given arguments."""
        pass


print("MCP Server base class defined!")

In [ ]:
# Example: Research Data MCP Server

class ResearchDataServer(MCPServer):
    """MCP server that exposes research datasets and analysis tools."""
    
    def __init__(self):
        super().__init__("research-data-server")
        
        # Simulated research datasets
        self._datasets = {
            "dataset://surveys/2024": {
                "name": "Public Opinion Survey 2024",
                "records": 1500,
                "columns": ["age", "gender", "opinion_score", "region"]
            },
            "dataset://experiments/llm-bias": {
                "name": "LLM Bias Study Results",
                "records": 500,
                "columns": ["model", "prompt_type", "bias_score", "confidence"]
            }
        }
        
        # Register resources
        for uri, data in self._datasets.items():
            self._resources[uri] = MCPResource(
                uri=uri,
                name=data["name"],
                description=f"Dataset with {data['records']} records",
                mime_type="application/json"
            )
        
        # Register tools
        self._tools["summarize_dataset"] = MCPTool(
            name="summarize_dataset",
            description="Get summary statistics for a dataset",
            input_schema={
                "type": "object",
                "properties": {
                    "dataset_uri": {"type": "string"}
                },
                "required": ["dataset_uri"]
            }
        )
        
        self._tools["run_analysis"] = MCPTool(
            name="run_analysis",
            description="Run statistical analysis on a dataset",
            input_schema={
                "type": "object",
                "properties": {
                    "dataset_uri": {"type": "string"},
                    "analysis_type": {
                        "type": "string",
                        "enum": ["correlation", "regression", "ttest"]
                    }
                },
                "required": ["dataset_uri", "analysis_type"]
            }
        )
    
    def read_resource(self, uri: str) -> str:
        """Read a dataset resource."""
        if uri not in self._datasets:
            raise ValueError(f"Resource not found: {uri}")
        return json.dumps(self._datasets[uri], indent=2)
    
    def call_tool(self, name: str, arguments: Dict[str, Any]) -> Any:
        """Call an analysis tool."""
        if name == "summarize_dataset":
            uri = arguments["dataset_uri"]
            if uri not in self._datasets:
                return {"error": f"Dataset not found: {uri}"}
            data = self._datasets[uri]
            return {
                "name": data["name"],
                "total_records": data["records"],
                "columns": data["columns"],
                "summary": f"Dataset contains {data['records']} records across {len(data['columns'])} variables"
            }
        
        elif name == "run_analysis":
            uri = arguments["dataset_uri"]
            analysis = arguments["analysis_type"]
            # Simulated analysis results
            return {
                "analysis_type": analysis,
                "dataset": uri,
                "result": f"Simulated {analysis} analysis completed",
                "statistics": {
                    "p_value": 0.03,
                    "effect_size": 0.45
                }
            }
        
        return {"error": f"Unknown tool: {name}"}


# Create and test the server
research_server = ResearchDataServer()

print("=== Server Info ===")
print(json.dumps(research_server.get_server_info(), indent=2))

print("\n=== Available Resources ===")
print(json.dumps(research_server.list_resources(), indent=2))

print("\n=== Available Tools ===")
print(json.dumps(research_server.list_tools(), indent=2))

In [ ]:
# Using the MCP Server

print("=== Reading a Resource ===")
resource_data = research_server.read_resource("dataset://surveys/2024")
print(resource_data)

print("\n=== Calling summarize_dataset Tool ===")
summary = research_server.call_tool("summarize_dataset", {
    "dataset_uri": "dataset://experiments/llm-bias"
})
print(json.dumps(summary, indent=2))

print("\n=== Running Analysis ===")
analysis = research_server.call_tool("run_analysis", {
    "dataset_uri": "dataset://surveys/2024",
    "analysis_type": "correlation"
})
print(json.dumps(analysis, indent=2))

## Part 3: Tool Use Safety and Best Practices

When enabling LLMs to use tools, safety considerations are crucial:

### Security Considerations

1. **Input Validation** - Always validate tool inputs before execution
2. **Sandboxing** - Run tools in isolated environments
3. **Rate Limiting** - Prevent abuse through excessive tool calls
4. **Audit Logging** - Log all tool invocations for review

### Best Practices

1. **Principle of Least Privilege** - Only expose necessary tools
2. **Clear Documentation** - Provide detailed tool descriptions
3. **Error Handling** - Return informative error messages
4. **Human-in-the-Loop** - Require approval for sensitive operations

### Common Pitfalls

- **Prompt Injection** - Malicious inputs that manipulate tool behavior
- **Information Leakage** - Tools exposing sensitive data
- **Infinite Loops** - Agents calling tools recursively
- **Resource Exhaustion** - Tools consuming excessive compute/memory

In [ ]:
# Safe Tool Wrapper with Logging and Validation

from datetime import datetime
from functools import wraps
import re

class SafeToolWrapper:
    """Wrapper that adds safety features to tool execution."""
    
    def __init__(self, tool: Tool, 
                 max_calls_per_minute: int = 10,
                 require_approval: bool = False):
        self.tool = tool
        self.max_calls = max_calls_per_minute
        self.require_approval = require_approval
        self.call_history: List[datetime] = []
        self.audit_log: List[Dict] = []
    
    def _check_rate_limit(self) -> bool:
        """Check if we're within rate limits."""
        now = datetime.now()
        # Remove calls older than 1 minute
        self.call_history = [
            t for t in self.call_history 
            if (now - t).seconds < 60
        ]
        return len(self.call_history) < self.max_calls
    
    def _validate_input(self, kwargs: Dict) -> bool:
        """Basic input validation."""
        for key, value in kwargs.items():
            if isinstance(value, str):
                # Check for potential injection patterns
                dangerous_patterns = [
                    r'__[a-z]+__',  # Python dunders
                    r'import\s+',   # Import statements
                    r'eval\s*\(',   # Eval calls
                    r'exec\s*\('    # Exec calls
                ]
                for pattern in dangerous_patterns:
                    if re.search(pattern, value, re.IGNORECASE):
                        return False
        return True
    
    def execute(self, **kwargs) -> Dict[str, Any]:
        """Safely execute the tool."""
        timestamp = datetime.now()
        
        # Check rate limit
        if not self._check_rate_limit():
            return {
                "success": False,
                "error": "Rate limit exceeded. Try again later."
            }
        
        # Validate input
        if not self._validate_input(kwargs):
            return {
                "success": False,
                "error": "Input validation failed. Potentially unsafe input detected."
            }
        
        # Check approval if required
        if self.require_approval:
            # In a real system, this would prompt for human approval
            print(f"[APPROVAL REQUIRED] Tool: {self.tool.name}, Args: {kwargs}")
        
        # Execute the tool
        try:
            result = self.tool.execute(**kwargs)
            success = True
            error = None
        except Exception as e:
            result = None
            success = False
            error = str(e)
        
        # Log the call
        self.call_history.append(timestamp)
        self.audit_log.append({
            "timestamp": timestamp.isoformat(),
            "tool": self.tool.name,
            "arguments": kwargs,
            "success": success,
            "error": error
        })
        
        return {
            "success": success,
            "result": result,
            "error": error
        }


# Wrap the calculator tool with safety features
safe_calculator = SafeToolWrapper(
    registry.get("calculator"),
    max_calls_per_minute=5
)

# Test normal usage
print("Normal calculation:")
print(safe_calculator.execute(expression="2 + 2"))

# Test potentially unsafe input
print("\nPotentially unsafe input:")
print(safe_calculator.execute(expression="__import__('os').system('ls')"))

# View audit log
print("\nAudit Log:")
for entry in safe_calculator.audit_log:
    print(json.dumps(entry, indent=2))

## Module 4 Exercises

### Exercise 1: Build a Tool-Using Agent
Create a simple agent that can:
- Accept natural language queries
- Decide which tool(s) to use
- Execute the tools and return results

You can use OpenAI, Anthropic, or any LLM API for the agent logic.

### Exercise 2: Implement an MCP Server
Build an MCP server that exposes:
- A resource (e.g., a research dataset or document collection)
- At least one tool (e.g., search, summarize, analyze)

Test the server with simulated MCP client requests.

### Exercise 3: Safety Analysis
For a tool of your choice, analyze:
- What could go wrong if the tool is misused?
- What safety measures would you implement?
- How would you test that the safety measures work?

Write a 1-page report on your analysis.

### Exercise 4: DSPy Pipeline (Advanced)
Using the DSPy framework (Khattab et al. 2023), create a pipeline that:
- Takes a research question as input
- Uses tools to gather relevant information
- Synthesizes a response

Compare the performance of manually-crafted prompts vs. DSPy-optimized prompts.